In [ ]:
# scraping script modified according to amazon.com
import scrapy
# Creating a new class to implement Spide


class AmazonReviewsSpider(scrapy.Spider):
    # Spider name
    name = 'amazon_reviews'
    # Domain names to scrape
    allowed_domains = ['amazon.com']
    # Base URL for the MacBook air reviews
    start_urls = []
    start_urls.append(
        'https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/product-reviews/B07N49F51N/ref=cm_cr_othr_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
    )
    myBaseUrl = 'https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/product-reviews/B07N49F51N/ref=cm_cr_getr_d_paging_btm_next_'
    add_on = '?ie=UTF8&reviewerType=all_reviews&pageNumber='
    # Creating list of urls to be scraped by appending page number a the end of base url
    for i in range(2, 20):
        start_urls.append(myBaseUrl + str(i) + add_on + str(i))
    # Defining a Scrapy parser

    def parse(self, response):
        data = response.css('#cm_cr-review_list')
        # Collecting product star ratings
        star_rating = data.css('.review-rating')
        # Collecting user reviews
        comments = data.css('.review-text')
        count = 0
        # Combining the results
        for review in star_rating:
            yield {
                'stars':
                ''.join(review.xpath('.//text()').extract()),
                'comment':
                ''.join(str(comments[count].xpath(".//text()").extract()))
            }
            count = count + 1

## Part1

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import Word
import nltk
# nltk.download('wordnet') # already downloaded
import spacy
# load spaCy model
nlp = spacy.load("en_core_web_sm")

In [2]:
df = pd.read_csv('reviews.csv')

In [3]:
print('Number of duplicate reviews: ', end='')
print('{}'.format(df[df.duplicated() == True].shape[0]))

Number of duplicate reviews: 0


In [4]:
# clean the comments
df['comment_clean'] = df['comment'].apply(lambda x: x.replace(r"'", ''))
df['comment_clean'] = df['comment_clean'].apply(lambda x: x.replace(r'"', ''))
df['comment_clean'] = df['comment_clean'].apply(
    lambda x: x.replace(r'[\n\n\n\n\n\n\n\n\n\n  \n  \n    , \n  ', ''))
df['comment_clean'] = df['comment_clean'].apply(
    lambda x: x.replace(r'\n, \n  \n]', ''))
# quantify the ratings
df['stars'] = df['stars'].replace(df['stars'].unique(), value=np.arange(1, 6))
# df clean
df.drop('comment', axis=1, inplace=True)

## Part2

In [5]:
# lowercase the comments
df['comment_clean'] = df['comment_clean'].apply(
    lambda x: ' '.join([x.lower() for x in x.split()]))
# remove punctuations
df['comment_clean'] = df['comment_clean'].str.replace('[^\w\s]', '')
# remove stopwords
stop = stopwords.words('english')
df['comment_clean'] = df['comment_clean'].apply(
    lambda x: ' '.join(x for x in x.split() if x not in stop))
# remove numbers
numbers = '0123456789'
df['comment_clean'] = df['comment_clean'].apply(
    lambda x: ''.join(x for x in list(x) if x not in numbers))
# stemming
st = PorterStemmer()
df['comment_clean'] = df['comment_clean'].apply(
    lambda x: " ".join([st.stem(word) for word in x.split()]))
# lemmatization
df['comment_clean'] = df['comment_clean'].apply(
    lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

## Part3

In [10]:
from collections import Counter
total_counts = []
for c in df['comment_clean']:
    doc = nlp(c)
    pos = []
    for tok in doc:
        pos.append(tok.pos_)
    counts = Counter(pos)
    total_counts.append(counts)

In [12]:
all_keys = []
for d in total_counts:
    for k, v in d.items():
        if k not in all_keys:
            all_keys.append(k)

In [14]:
for key in all_keys:
    df[key] = 0

In [19]:
for i, d in enumerate(total_counts):
    for k, v in d.items():
        df.loc[i, k] = v
    

In [21]:
df

,stars,comment_clean,PROPN,SCONJ,NOUN,VERB,NUM,ADJ,ADV,ADP,AUX,PART,PRON,INTJ,X,DET,CCONJ,SYM
0,1,purchas laptop upgrad ram gb ssd tb arriv near...,26,2,17,17,4,6,3,1,0,0,0,0,0,0,0,0
1,1,item came promptli begin touch screen didnt wo...,2,0,12,12,0,1,1,0,1,3,0,0,0,0,0,0
2,1,alreadi issu requir repair beyond inhous guy,2,0,3,0,0,1,0,1,0,0,0,0,0,0,0,0
3,2,came even around pm thank usp still work scrat...,3,1,18,14,0,7,5,2,1,1,1,0,0,0,0,0
4,3,laptop ok price lack usbc problem especi power...,7,0,8,2,0,3,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,4,happi product,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
106,4,best laptop,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
107,5,dc adapt plugin tight fit easili becom loo let...,13,0,11,7,1,7,2,0,0,0,0,0,0,0,0,0
108,3,second dell laptop first one great replac get ...,7,0,14,8,2,7,2,1,0,1,0,0,1,0,0,0
